# 파일 이름 및 변수

## 최고할인률.csv
- 게임 이름을 기준으로 최고할인률과 기간 데이터
- 기간 데이터를 시작일과 끝나는 일로 나눔

## 접속자수.csv
- 게임접속자수1.csv

## 메타크리틱점수.csv
- 게임이름별 메타크리틱 점수 + 유저 점수

## 게임데이터.csv
- 게임들의 여러 데이터 

## 1. 최고할인률 정제
- 할인률과 할인기간에 누락값 존재
- 중복된 게임명 존재 -> 게임명을 기준으로 중복값 삭제

In [1]:
import pandas as pd
df = pd.read_csv('최고할인률.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19426 entries, 0 to 19425
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임명     19389 non-null  object
 1   할인률     17484 non-null  object
 2   할인기간    17484 non-null  object
dtypes: object(3)
memory usage: 455.4+ KB


In [5]:
df[df['할인률'].isna()]

,게임명,할인률,할인기간
0,guilty gear -strive-,NaN,NaN
107,트리 오브 세이비어,NaN,NaN
263,call of duty?: modern warfare? 3 (2011),NaN,NaN
264,call of duty?: modern warfare? 3 (2011),NaN,NaN
304,ホロライブごろごろマウンテン,NaN,NaN
...,...,...,...
19407,식민지 고양이: 위기의 한가운데에서,NaN,NaN
19408,雪花斗?棋,NaN,NaN
19413,pitfight,NaN,NaN
19416,이상한 블록36,NaN,NaN


In [7]:
df['게임명'].duplicated().sum()

np.int64(585)

In [8]:
df[df['게임명'].duplicated()]

,게임명,할인률,할인기간
78,fast food simulator,81,2020.11.12 ~ 2025.03.20
245,shift happens,90,2017.06.23 ~ 2025.07.25
264,call of duty?: modern warfare? 3 (2011),NaN,NaN
582,small world,69,2020.10.22 ~
705,one life,90,2023.01.17 ~ 2024.12.05
...,...,...,...
17037,retro racers,30,2022.10.28 ~ 2025.07.11
17041,summer camp,20,2024.06.28 ~ 2025.07.11
18484,??,10,2025.05.16 ~ 2025.07.11
18707,探???-小?·ctrl女?（附送29999??）,78,2019.03.30 ~ 2025.02.18


### 중복 삭제

In [11]:
df[df['게임명'].duplicated()]

,게임명,할인률,할인기간
78,fast food simulator,81,2020.11.12 ~ 2025.03.20
245,shift happens,90,2017.06.23 ~ 2025.07.25
264,call of duty?: modern warfare? 3 (2011),NaN,NaN
582,small world,69,2020.10.22 ~
705,one life,90,2023.01.17 ~ 2024.12.05
...,...,...,...
17037,retro racers,30,2022.10.28 ~ 2025.07.11
17041,summer camp,20,2024.06.28 ~ 2025.07.11
18484,??,10,2025.05.16 ~ 2025.07.11
18707,探???-小?·ctrl女?（附送29999??）,78,2019.03.30 ~ 2025.02.18


In [13]:
df.shape

(19426, 3)

In [14]:
df.drop_duplicates(subset=['게임명'], inplace=True)

In [15]:
df

,게임명,할인률,할인기간
0,guilty gear -strive-,NaN,NaN
1,magic: the gathering arena,가격 정보가 없습니다.,세일 정보가 없습니다.
2,ez2on reboot : r,47,2021.03.18 ~ 2025.07.11
3,sons of the forest,66,2023.06.30 ~ 2025.07.11
4,left 4 dead 2,90,2017.06.23 ~ 2025.07.11
...,...,...,...
19421,crabs and rats,60,2017.04.11 ~ 2025.07.11
19422,adventure jam,40,2019.02.26 ~ 2019.03.05
19423,the train arrived!,70,2018.07.07 ~ 2021.11.02
19424,dispatch control,50,2021.04.30 ~


In [17]:
df['게임명'].duplicated().sum()

np.int64(0)

### 누락값 처리
- 할인률 누락이나, 가격정보가 없는 것들은 전부 0으로 처리
- 할인률이 음수인 데이터 2개 확인, 게임명 확인후 웹사이트에서 보고 수기 입력

- 게임명이 누락된 값 1개 발견, 수기 입력 -> 해당 게임아이디로 검색이 안됨 -> 행 삭제

- 할인기간은 할인시작일과 종료일로 나눔
- 누락값은 세일 정보가 없습니다로 통일
- 할인 시작 기간만 있을 경우에는 종료일에 진행중

In [19]:
df

,게임명,할인률,할인기간
0,guilty gear -strive-,NaN,NaN
1,magic: the gathering arena,가격 정보가 없습니다.,세일 정보가 없습니다.
2,ez2on reboot : r,47,2021.03.18 ~ 2025.07.11
3,sons of the forest,66,2023.06.30 ~ 2025.07.11
4,left 4 dead 2,90,2017.06.23 ~ 2025.07.11
...,...,...,...
19421,crabs and rats,60,2017.04.11 ~ 2025.07.11
19422,adventure jam,40,2019.02.26 ~ 2019.03.05
19423,the train arrived!,70,2018.07.07 ~ 2021.11.02
19424,dispatch control,50,2021.04.30 ~


In [18]:
df[df['할인률'].isna()]

,게임명,할인률,할인기간
0,guilty gear -strive-,NaN,NaN
107,트리 오브 세이비어,NaN,NaN
263,call of duty?: modern warfare? 3 (2011),NaN,NaN
304,ホロライブごろごろマウンテン,NaN,NaN
434,guilty gear xrd -sign-,NaN,NaN
...,...,...,...
19407,식민지 고양이: 위기의 한가운데에서,NaN,NaN
19408,雪花斗?棋,NaN,NaN
19413,pitfight,NaN,NaN
19416,이상한 블록36,NaN,NaN


In [21]:
df['할인률'].fillna(0, inplace=True)

In [26]:
df[df['할인률'].isna()]

,게임명,할인률,할인기간


In [28]:
df['할인률'].value_counts()

할인률
50          3591
0           1923
80          1312
75          1185
90          1138
            ... 
98             3
-1039900       2
8              2
4              1
14             1
Name: count, Length: 96, dtype: int64

In [29]:
df['할인률'].value_counts(dropna=False)

할인률
50          3591
0           1923
80          1312
75          1185
90          1138
            ... 
98             3
-1039900       2
8              2
4              1
14             1
Name: count, Length: 96, dtype: int64

In [42]:
df[df['할인률'].str.contains('정보', na=False)]

,게임명,할인률,할인기간
1,magic: the gathering arena,가격 정보가 없습니다.,세일 정보가 없습니다.
11,dota 2,가격 정보가 없습니다.,세일 정보가 없습니다.
18,monster hunter: world,가격 정보가 없습니다.,세일 정보가 없습니다.
35,goose goose duck,가격 정보가 없습니다.,세일 정보가 없습니다.
40,fragpunk,가격 정보가 없습니다.,세일 정보가 없습니다.
...,...,...,...
19287,call of duty?: black ops ii,가격 정보가 없습니다.,세일 정보가 없습니다.
19294,"damage inc f4u-4 ""reaper"" corsair",가격 정보가 없습니다.,세일 정보가 없습니다.
19295,continent of the ninth seal,가격 정보가 없습니다.,세일 정보가 없습니다.
19368,empires mod,가격 정보가 없습니다.,세일 정보가 없습니다.


In [43]:
# "정보"가 포함된 행 찾기 (결측값은 False 처리)
mask = df['할인률'].str.contains('정보', na=False)

# 해당 인덱스의 '할인률' 값을 0으로 변경
df.loc[mask, '할인률'] = 0

In [45]:
df['할인률'].str.contains('정보', na=False).sum()

np.int64(0)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18841 entries, 0 to 19425
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임명     18840 non-null  object
 1   할인률     18841 non-null  object
 2   할인기간    16918 non-null  object
dtypes: object(3)
memory usage: 588.8+ KB


In [50]:
df['할인률'] = df['할인률'].astype(int)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18841 entries, 0 to 19425
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임명     18840 non-null  object
 1   할인률     18841 non-null  int64 
 2   할인기간    16918 non-null  object
dtypes: int64(1), object(2)
memory usage: 588.8+ KB


In [37]:
df[df['게임명'].isna()]

,게임명,할인률,할인기간
10931,NaN,75,2019.03.05 ~ 2025.01.03


In [52]:
asd = pd.read_csv('02-3. steam_data_real.csv')
asd[10931:10932]

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수
10931,418010,NaN,NaN,어드벤처 | 캐주얼 | 액션 | 인디 | 시뮬레이션,Windows,0,NaN,NaN


In [54]:
df.drop(10931, inplace=True)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18840 entries, 0 to 19425
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임명     18840 non-null  object
 1   할인률     18840 non-null  int64 
 2   할인기간    16917 non-null  object
dtypes: int64(1), object(2)
memory usage: 588.8+ KB


In [56]:
df[df['할인기간'].isna()]

,게임명,할인률,할인기간
0,guilty gear -strive-,0,NaN
107,트리 오브 세이비어,0,NaN
263,call of duty?: modern warfare? 3 (2011),0,NaN
304,ホロライブごろごろマウンテン,0,NaN
434,guilty gear xrd -sign-,0,NaN
...,...,...,...
19407,식민지 고양이: 위기의 한가운데에서,0,NaN
19408,雪花斗?棋,0,NaN
19413,pitfight,0,NaN
19416,이상한 블록36,0,NaN


In [59]:
df['할인기간'].fillna('세일 정보가 없습니다.', inplace=True)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18840 entries, 0 to 19425
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임명     18840 non-null  object
 1   할인률     18840 non-null  int64 
 2   할인기간    18840 non-null  object
dtypes: int64(1), object(2)
memory usage: 588.8+ KB


In [62]:
df

,게임명,할인률,할인기간
0,guilty gear -strive-,0,세일 정보가 없습니다.
1,magic: the gathering arena,0,세일 정보가 없습니다.
2,ez2on reboot : r,47,2021.03.18 ~ 2025.07.11
3,sons of the forest,66,2023.06.30 ~ 2025.07.11
4,left 4 dead 2,90,2017.06.23 ~ 2025.07.11
...,...,...,...
19421,crabs and rats,60,2017.04.11 ~ 2025.07.11
19422,adventure jam,40,2019.02.26 ~ 2019.03.05
19423,the train arrived!,70,2018.07.07 ~ 2021.11.02
19424,dispatch control,50,2021.04.30 ~


In [63]:
# 할인 시작일과 종료일을 저장할 새 컬럼 생성
df['할인 시작일'] = ''
df['할인 종료일'] = ''

# 조건 분기 처리
for idx, period in df['할인기간'].items():
    if '세일 정보가 없습니다' in period:
        df.at[idx, '할인 시작일'] = '세일 정보가 없습니다.'
        df.at[idx, '할인 종료일'] = '세일 정보가 없습니다.'
    elif '~' in period:
        parts = period.split('~')
        start_date = parts[0].strip()
        end_date = parts[1].strip() if len(parts) > 1 and parts[1].strip() else '진행중'

        df.at[idx, '할인 시작일'] = start_date
        df.at[idx, '할인 종료일'] = end_date
    else:
        df.at[idx, '할인 시작일'] = '세일 정보가 없습니다.'
        df.at[idx, '할인 종료일'] = '세일 정보가 없습니다.'


In [64]:
df

,게임명,할인률,할인기간,할인 시작일,할인 종료일
0,guilty gear -strive-,0,세일 정보가 없습니다.,세일 정보가 없습니다.,세일 정보가 없습니다.
1,magic: the gathering arena,0,세일 정보가 없습니다.,세일 정보가 없습니다.,세일 정보가 없습니다.
2,ez2on reboot : r,47,2021.03.18 ~ 2025.07.11,2021.03.18,2025.07.11
3,sons of the forest,66,2023.06.30 ~ 2025.07.11,2023.06.30,2025.07.11
4,left 4 dead 2,90,2017.06.23 ~ 2025.07.11,2017.06.23,2025.07.11
...,...,...,...,...,...
19421,crabs and rats,60,2017.04.11 ~ 2025.07.11,2017.04.11,2025.07.11
19422,adventure jam,40,2019.02.26 ~ 2019.03.05,2019.02.26,2019.03.05
19423,the train arrived!,70,2018.07.07 ~ 2021.11.02,2018.07.07,2021.11.02
19424,dispatch control,50,2021.04.30 ~,2021.04.30,진행중


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18840 entries, 0 to 19425
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임명     18840 non-null  object
 1   할인률     18840 non-null  int64 
 2   할인기간    18840 non-null  object
 3   할인 시작일  18840 non-null  object
 4   할인 종료일  18840 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.4+ MB


In [69]:
df.to_csv('최종_할인률.csv', index=False)

In [70]:
df['할인률'].value_counts()

할인률
 50         3591
 0          2397
 80         1312
 75         1184
 90         1138
            ... 
 98            3
-1039900       2
 8             2
 4             1
 14            1
Name: count, Length: 95, dtype: int64

In [75]:
df

,게임명,할인률,할인기간,할인 시작일,할인 종료일
0,guilty gear -strive-,0,세일 정보가 없습니다.,세일 정보가 없습니다.,세일 정보가 없습니다.
1,magic: the gathering arena,0,세일 정보가 없습니다.,세일 정보가 없습니다.,세일 정보가 없습니다.
2,ez2on reboot : r,47,2021.03.18 ~ 2025.07.11,2021.03.18,2025.07.11
3,sons of the forest,66,2023.06.30 ~ 2025.07.11,2023.06.30,2025.07.11
4,left 4 dead 2,90,2017.06.23 ~ 2025.07.11,2017.06.23,2025.07.11
...,...,...,...,...,...
19421,crabs and rats,60,2017.04.11 ~ 2025.07.11,2017.04.11,2025.07.11
19422,adventure jam,40,2019.02.26 ~ 2019.03.05,2019.02.26,2019.03.05
19423,the train arrived!,70,2018.07.07 ~ 2021.11.02,2018.07.07,2021.11.02
19424,dispatch control,50,2021.04.30 ~,2021.04.30,진행중


In [73]:
df[df['할인률'] == -1039900]

,게임명,할인률,할인기간,할인 시작일,할인 종료일
17454,war dust vr : storm | 32vs32 battlefields,-1039900,2018.10.20 ~,2018.10.20,진행중
18614,war dust vr: 32v32 battlefields,-1039900,2018.10.20 ~,2018.10.20,진행중


In [76]:
df.loc[17454, '할인률'] = 0
df.loc[17454, '할인기간'] = '세일 정보가 없습니다.'
df.loc[17454, '할인 시작일'] = '세일 정보가 없습니다.'
df.loc[17454, '할인 종료일'] = '세일 정보가 없습니다.'

In [77]:
df[df['할인률'] == -1039900]

,게임명,할인률,할인기간,할인 시작일,할인 종료일
18614,war dust vr: 32v32 battlefields,-1039900,2018.10.20 ~,2018.10.20,진행중


In [78]:
df.loc[18614, '할인률'] = 60

In [80]:
df['할인률'].describe()

count    18840.000000
mean        48.883174
std         28.261894
min          0.000000
25%         28.000000
50%         50.000000
75%         75.000000
max        100.000000
Name: 할인률, dtype: float64

In [83]:
df[df['할인 종료일'] == '진행중']

,게임명,할인률,할인기간,할인 시작일,할인 종료일
5,human fall flat,50,2024.11.01 ~,2024.11.01,진행중
6,riders republic,90,2023.06.09 ~,2023.06.09,진행중
9,granblue fantasy versus: rising,61,2024.02.20 ~,2024.02.20,진행중
12,crime simulator,25,2025.06.18 ~,2025.06.18,진행중
21,a way out,90,2020.06.19 ~,2020.06.19,진행중
...,...,...,...,...,...
19400,the snake kingdom of gennibar-six,80,2018.05.19 ~,2018.05.19,진행중
19409,打?板 floorbreaker,10,2025.08.01 ~,2025.08.01,진행중
19412,oops! my king!,71,2023.05.30 ~,2023.05.30,진행중
19414,twin flames,80,2019.08.29 ~,2019.08.29,진행중


현재 최종할인률 데이터가 정확하지 않은 문제 -> 다시 크롤링

## 2. 게임별 평균 접속자수 정제
- 열 이름 한글로 변환 ex) 2308_peak
- 열 이름 순서 연도별로 변경

In [119]:
df = pd.read_csv('게임접속자수1.csv')
df

,Unnamed: 0,id,2023 November peak,2024 February peak,2025 February avg,2024 December avg,2024 August peak,2024 January peak,2025 January peak,2025 May peak,...,2024 July avg,2025 January avg,2024 July peak,2024 November avg,2025 March avg,2024 February avg,2023 September avg,2025 February peak,2025 April avg,2024 April peak
0,0,1384160,2747.0,2503.0,1523.3,1848.9,4231.0,3493.0,2494.0,10191.0,...,2871.5,1567.2,6794.0,3299.6,1714.5,1632.2,2183.7,2417.0,1836.7,4181.0
1,1,2141910,7357.0,7573.0,6148.8,5387.2,9627.0,5450.0,7679.0,9083.0,...,5098.8,5368.5,12058.0,6499.9,5850.9,4319.1,3762.1,11048.0,6627.6,8543.0
2,2,1477590,637.0,325.0,93.5,89.0,419.0,272.0,731.0,646.0,...,131.4,115.2,1143.0,99.1,95.6,98.9,94.8,242.0,78.2,324.0
3,3,1326470,17207.0,53964.0,9338.0,16985.8,13890.0,17076.0,44610.0,13846.0,...,8765.7,17323.0,18357.0,13790.0,11098.6,11038.0,5583.1,20587.0,12970.8,20005.0
4,4,550,71345.0,35794.0,28121.3,27355.1,34348.0,58686.0,41486.0,50899.0,...,27361.6,26582.6,48914.0,30372.5,24513.3,22900.6,26449.6,51703.0,20885.2,35358.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,5556,1172650,17.0,12.0,3.9,6.9,20.0,14.0,16.0,21.0,...,6.7,5.4,24.0,4.4,19.3,3.3,3.5,13.0,8.5,38.0
5557,5557,718590,18.0,15.0,4.0,4.6,8.0,15.0,18.0,7.0,...,3.5,5.2,10.0,7.5,3.2,5.2,9.2,15.0,3.5,35.0
5558,5558,2964020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5559,5559,3266210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
import pandas as pd

# 월 영어 → 숫자 변환용 딕셔너리
month_map = {
    'January': '01', 'February': '02', 'March': '03', 'April': '04',
    'May': '05', 'June': '06', 'July': '07', 'August': '08',
    'September': '09', 'October': '10', 'November': '11', 'December': '12'
}

def convert_column_name(col):
    if col in ['Unnamed: 0', 'id']:
        return col  # 그대로 유지
    try:
        parts = col.split()  # ['2023', 'December', 'peak']
        year = parts[0][2:]  # '23'
        month = month_map[parts[1]]  # '12'
        kind = parts[2].capitalize()  # 'Peak' 또는 'Avg'
        return f"{year}{month}_{kind}"
    except:
        return col  # 혹시 오류 나면 원래 이름 유지

# 컬럼 이름 변경
df.columns = [convert_column_name(col) for col in df.columns]
df

,Unnamed: 0,id,2311_Peak,2402_Peak,2502_Avg,2412_Avg,2408_Peak,2401_Peak,2501_Peak,2505_Peak,...,2407_Avg,2501_Avg,2407_Peak,2411_Avg,2503_Avg,2402_Avg,2309_Avg,2502_Peak,2504_Avg,2404_Peak
0,0,1384160,2747.0,2503.0,1523.3,1848.9,4231.0,3493.0,2494.0,10191.0,...,2871.5,1567.2,6794.0,3299.6,1714.5,1632.2,2183.7,2417.0,1836.7,4181.0
1,1,2141910,7357.0,7573.0,6148.8,5387.2,9627.0,5450.0,7679.0,9083.0,...,5098.8,5368.5,12058.0,6499.9,5850.9,4319.1,3762.1,11048.0,6627.6,8543.0
2,2,1477590,637.0,325.0,93.5,89.0,419.0,272.0,731.0,646.0,...,131.4,115.2,1143.0,99.1,95.6,98.9,94.8,242.0,78.2,324.0
3,3,1326470,17207.0,53964.0,9338.0,16985.8,13890.0,17076.0,44610.0,13846.0,...,8765.7,17323.0,18357.0,13790.0,11098.6,11038.0,5583.1,20587.0,12970.8,20005.0
4,4,550,71345.0,35794.0,28121.3,27355.1,34348.0,58686.0,41486.0,50899.0,...,27361.6,26582.6,48914.0,30372.5,24513.3,22900.6,26449.6,51703.0,20885.2,35358.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,5556,1172650,17.0,12.0,3.9,6.9,20.0,14.0,16.0,21.0,...,6.7,5.4,24.0,4.4,19.3,3.3,3.5,13.0,8.5,38.0
5557,5557,718590,18.0,15.0,4.0,4.6,8.0,15.0,18.0,7.0,...,3.5,5.2,10.0,7.5,3.2,5.2,9.2,15.0,3.5,35.0
5558,5558,2964020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5559,5559,3266210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [125]:
import re

def sort_key(col):
    if col == 'id':
        return (0, 0, 0)  # 'id'는 가장 먼저

    match = re.match(r'(\d{2})(\d{2})_(Peak|Avg)', col)
    if match:
        year = int(match.group(1))     # '23', '24', '25' → int
        month = int(match.group(2))    # '01' ~ '12' → int
        kind = 0 if match.group(3) == 'Peak' else 1  # Peak 먼저
        return (year, month, kind)
    else:
        return (99, 99, 99)  # 매칭 안 되는 건 맨 뒤로

# 정렬된 컬럼 리스트 만들기
sorted_columns = sorted(df.columns, key=sort_key)

# 정렬된 열 순서 적용
df = df[sorted_columns]
df

,id,2308_Peak,2308_Avg,2309_Peak,2309_Avg,2310_Peak,2310_Avg,2311_Peak,2311_Avg,2312_Peak,...,2503_Peak,2503_Avg,2504_Peak,2504_Avg,2505_Peak,2505_Avg,2506_Peak,2506_Avg,2507_Peak,2507_Avg
0,1384160,12512.0,2640.3,3943.0,2183.7,2427.0,1552.0,2747.0,1486.7,12555.0,...,5963.0,1714.5,2890.0,1836.7,10191.0,2281.1,4058.0,2038.7,2320.0,1546.2
1,2141910,4075.0,2898.4,6793.0,3762.1,5277.0,3675.6,7357.0,3909.8,5124.0,...,8358.0,5850.9,12119.0,6627.6,9083.0,6411.9,17989.0,8669.7,16239.0,8644.6
2,1477590,208.0,80.3,849.0,94.8,509.0,112.5,637.0,111.6,419.0,...,252.0,95.6,211.0,78.2,646.0,107.4,335.0,90.5,281.0,85.3
3,1326470,13079.0,6717.4,11412.0,5583.1,17680.0,7321.7,17207.0,8110.6,16612.0,...,34881.0,11098.6,39476.0,12970.8,13846.0,6460.2,49540.0,7913.1,54093.0,18785.9
4,550,37223.0,24374.2,78384.0,26449.6,45432.0,22070.2,71345.0,22509.4,56599.0,...,43139.0,24513.3,36296.0,20885.2,50899.0,22194.5,52111.0,26652.2,41687.0,26208.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,1172650,16.0,5.1,14.0,3.5,14.0,3.6,17.0,4.0,13.0,...,76.0,19.3,29.0,8.5,21.0,6.4,17.0,5.4,32.0,7.8
5557,718590,12.0,3.6,41.0,9.2,18.0,5.0,18.0,5.3,12.0,...,11.0,3.2,9.0,3.5,7.0,2.6,11.0,4.2,11.0,3.1
5558,2964020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5559,3266210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5561 entries, 0 to 5560
Data columns (total 49 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         5561 non-null   int64  
 1   2308_Peak  5561 non-null   float64
 2   2308_Avg   5561 non-null   float64
 3   2309_Peak  5561 non-null   float64
 4   2309_Avg   5561 non-null   float64
 5   2310_Peak  5561 non-null   float64
 6   2310_Avg   5561 non-null   float64
 7   2311_Peak  5561 non-null   float64
 8   2311_Avg   5561 non-null   float64
 9   2312_Peak  5561 non-null   float64
 10  2312_Avg   5561 non-null   float64
 11  2401_Peak  5561 non-null   float64
 12  2401_Avg   5561 non-null   float64
 13  2402_Peak  5561 non-null   float64
 14  2402_Avg   5561 non-null   float64
 15  2403_Peak  5561 non-null   float64
 16  2403_Avg   5561 non-null   float64
 17  2404_Peak  5561 non-null   float64
 18  2404_Avg   5561 non-null   float64
 19  2405_Peak  5561 non-null   float64
 20  2405_Avg

In [127]:
df.duplicated().sum()

np.int64(0)

In [128]:
df.describe()

,id,2308_Peak,2308_Avg,2309_Peak,2309_Avg,2310_Peak,2310_Avg,2311_Peak,2311_Avg,2312_Peak,...,2503_Peak,2503_Avg,2504_Peak,2504_Avg,2505_Peak,2505_Avg,2506_Peak,2506_Avg,2507_Peak,2507_Avg
count,5.561000e+03,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,...,5.561000e+03,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000,5561.000000
mean,1.573406e+06,1121.218486,559.196961,1111.367380,508.971642,1137.169394,511.080543,1083.527063,494.306006,1181.932206,...,1.341121e+03,598.807912,1262.919079,586.281154,1203.705269,574.882431,1326.902176,619.966319,1257.558712,609.154774
std,9.976078e+05,19259.118048,9709.514666,16865.308155,8242.340959,16105.435242,7760.934295,15396.538974,7551.491408,16771.672234,...,2.213231e+04,8414.295539,17646.734839,8128.332245,16059.782445,7877.925217,16268.945875,8099.472916,17402.123401,8029.881710
min,2.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.052100e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.439750e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.336550e+06,34.000000,10.900000,34.000000,10.700000,36.000000,10.900000,41.000000,12.600000,44.000000,...,6.300000e+01,22.800000,61.000000,22.100000,68.000000,23.600000,72.000000,24.900000,70.000000,26.200000
max,3.860210e+06,875343.000000,450982.300000,777466.000000,437273.400000,780443.000000,435621.000000,767353.000000,433727.600000,802728.000000,...,1.329605e+06,402851.200000,861598.000000,393173.200000,804674.000000,408987.700000,778327.000000,411123.300000,944484.000000,426672.700000


In [129]:
df.to_csv('최종_접속자수.csv', index=False)

## 3. 메타크리틱 점수 정제

In [130]:
df = pd.read_csv('메타크리틱1차.csv')
df

,게임명,meta_score,user_score
0,guilty-gear-strive,NaN,NaN
1,magic-the-gathering-arena,84,3.1
2,ez2on-reboot-r,tbd,tbd
3,sons-of-the-forest,86,7.3
4,left-4-dead-2,89,8.2
...,...,...,...
18816,crabs-and-rats,tbd,tbd
18817,adventure-jam,tbd,tbd
18818,the-train-arrived,tbd,tbd
18819,dispatch-control,tbd,tbd


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18821 entries, 0 to 18820
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   게임명         18819 non-null  object
 1   meta_score  14846 non-null  object
 2   user_score  14802 non-null  object
dtypes: object(3)
memory usage: 441.2+ KB


# 최종 정제 파일
- 최종_접속자수.csv
- 최종_스팀게임정보.csv
- 최종_유저데이터.csv